**下面是測試結果：**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

In [ ]:
# loading the stored model from file
model=load_model(r'/content/drive/Shareddrives/AAML_final/FireNet-LightWeight-Network-for-Fire-Detection/Trained_Model_FIreNet/train1.h5', compile=False)

cap = cv2.VideoCapture(r'/content/drive/Shareddrives/AAML_final/Our_Complete_Dataset_Video_and_extra_NoFire_Frames/fire/FireVid20.mp4')
time.sleep(2)

if cap.isOpened(): # try to get the first frame
    rval, frame = cap.read()
else:
    rval = False


IMG_SIZE = 64

In [ ]:
import os
import psutil

while(1):

    rval, image = cap.read()
    if rval==True:
        orig = image.copy()
        
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))  
        image = image.astype("float") / 255.0
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        
        tic = time.time()
        fire_prob = model.predict(image)[0][0] * 100
        toc = time.time()
        # CPU usage
        print('CPU usage :', psutil.cpu_percent(), '%')
        # Memory usage
        total_memory, used_memory, free_memory = map(
        int, os.popen('free -t -m').readlines()[-1].split()[1:])
        print("RAM memory: ", round((used_memory/total_memory) * 100, 2), '%')
        print("Time taken = ", toc - tic)
        print("FPS: ", 1 / np.float64(toc - tic))
        print("Fire Probability: ", fire_prob)
        print(image.shape)
        
        label = "Fire Probability: " + str(fire_prob)
        cv2.putText(orig, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 255, 0), 2)

        cv2_imshow(orig)
        
        key = cv2.waitKey(10)
        if key == 27: # exit on ESC
            break
    elif rval==False:
            break
end = time.time()


cap.release()
cv2.destroyAllWindows()